In [ ]:
!pip install tensorflow

In [ ]:
pip install tensorflow pillow


In [ ]:
import zipfile
from google.colab import files

# Upload zipped folders
uploaded = files.upload()

# Extract the Drowsy and Non Drowsy folders
for filename in uploaded.keys():
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall()

Saving converted_keras.zip to converted_keras.zip
Saving converted_savedmodel.zip to converted_savedmodel.zip


In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from PIL import Image, ImageOps

# Load the model
model = load_model("/content/keras_model.h5", compile=False)

# Load the labels
class_names = open("/content/labels.txt", "r").readlines()

# Initialize the webcam
camera = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = camera.read()
    if not ret:
        break

    # Convert the frame to RGB
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Resize the image to 224x224 pixels
    image = Image.fromarray(image)
    image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
    image_array = np.asarray(image)

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Create a batch of one image
    data = np.expand_dims(normalized_image_array, axis=0)

    # Perform prediction
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index].strip()
    confidence_score = prediction[0][index]

    # Display the resulting frame
    cv2.putText(frame, f"{class_name}: {confidence_score:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('Drowsiness Detection', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
camera.release()
cv2.destroyAllWindows()


TypeError: Error when deserializing class 'DepthwiseConv2D' using config={'name': 'expanded_conv_depthwise', 'trainable': True, 'dtype': 'float32', 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'linear', 'use_bias': False, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'bias_regularizer': None, 'activity_regularizer': None, 'bias_constraint': None, 'depth_multiplier': 1, 'depthwise_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'depthwise_regularizer': None, 'depthwise_constraint': None}.

Exception encountered: Unrecognized keyword arguments passed to DepthwiseConv2D: {'groups': 1}

In [ ]:
from google.colab import files

# Upload the alert sound file
uploaded = files.upload()

# Get the file name
alert_sound_file = list(uploaded.keys())[0]

Saving alert.wav to alert.wav


In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from PIL import Image, ImageOps
from google.colab.patches import cv2_imshow
from google.colab import output
import base64
import io
from IPython.display import display, Javascript, Audio

# Load the model
model = load_model("/content/keras_model.h5", compile=False)

# Load the labels
class_names = open("/content/labels.txt", "r").readlines()

# Global variables
drowsy_frame_count = 0
ALERT_THRESHOLD = 5  # Number of continuous drowsy frames to trigger alert
alert_sound_file = "/content/alert.wav"  # Path to the alert sound file

def capture_image():
    """
    Function to capture an image from the webcam via JavaScript.
    """
    js = Javascript('''
        async function takePhoto() {
            const div = document.createElement('div');
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const button = document.createElement('button');
            button.textContent = 'Stop';

            div.appendChild(video);
            div.appendChild(button);
            document.body.appendChild(div);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            async function captureFrame() {
                if (!video.srcObject.active) {
                    return null;
                }
                canvas.width = video.videoWidth;
                canvas.height = video.videoHeight;
                canvas.getContext('2d').drawImage(video, 0, 0);
                return canvas.toDataURL('image/jpeg').split(',')[1];
            }

            button.onclick = () => {
                stream.getTracks().forEach(track => track.stop());
                div.remove();
            };

            while (video.srcObject.active) {
                const image = await captureFrame();
                google.colab.kernel.invokeFunction('notebook.captureFrameCallback', [image], {});
                await new Promise(resolve => setTimeout(resolve, 1000)); // Capture every second
            }
        }
        takePhoto();
    ''')
    display(js)

def process_frame(frame):
    """
    Process the frame and make a prediction.
    """
    global drowsy_frame_count

    # Resize the frame to 224x224 pixels
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
    image_array = np.asarray(image)

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Create a batch of one image
    data = np.expand_dims(normalized_image_array, axis=0)

    # Perform prediction
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index].strip()
    confidence_score = prediction[0][index]

    # Check if the class is "drowsy"
    if class_name == "drowsy":
        drowsy_frame_count += 1
    else:
        drowsy_frame_count = 0  # Reset the counter if not drowsy

    # Trigger alert if continuous drowsy frames exceed the threshold
    if drowsy_frame_count >= ALERT_THRESHOLD:
        display(Audio(alert_sound_file, autoplay=True))
        drowsy_frame_count = 0  # Reset the counter after alert

    return class_name, confidence_score

def capture_frame_callback(image_data):
    """
    Callback to process the captured image.
    """
    global drowsy_frame_count

    if image_data is None:
        return

    # Decode the image data
    image_bytes = base64.b64decode(image_data)
    frame = Image.open(io.BytesIO(image_bytes))
    frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)

    # Process the frame
    class_name, confidence_score = process_frame(frame)

    # Display the result
    cv2.putText(frame, f"{class_name}: {confidence_score:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2_imshow(frame)

# Register callback function
output.register_callback('notebook.captureFrameCallback', capture_frame_callback)

# Start capturing
capture_image()

<IPython.core.display.Javascript object>

In [ ]:
!pip install opencv-python


In [ ]:
pip install tensorflow==2.12


In [ ]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!ffmpeg -i /content/alert.wav /content/alert.mp3

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [ ]:
from IPython.display import Audio

# Play the converted audio file
Audio("/content/alert.mp3", autoplay=True)

In [ ]:
def process_frame(self, frame):
    """
    Process the frame and make a prediction.
    """
    # Resize the frame to 224x224 pixels
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
    image_array = np.asarray(image)

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Create a batch of one image
    data = np.expand_dims(normalized_image_array, axis=0)

    # Perform prediction
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index].strip()
    confidence_score = prediction[0][index]

    # Check if the class is "drowsy"
    if class_name == "drowsy":
        self.drowsy_frame_count += 1
    else:
        self.drowsy_frame_count = 0  # Reset the counter if not drowsy

    # Debug print to check predictions and drowsy frame count
    print(f"Class: {class_name}, Confidence: {confidence_score:.2f}, Drowsy Frames: {self.drowsy_frame_count}")

    # Trigger alert if continuous drowsy frames exceed the threshold
    if self.drowsy_frame_count >= self.ALERT_THRESHOLD:
        print("ALERT: Drowsiness detected! Playing alert sound...")
        display(Audio(alert_sound_file, autoplay=True))
        self.drowsy_frame_count = 0  # Reset the counter after alert

    return class_name, confidence_score

In [ ]:
def capture_frame_callback(image_data):
    """
    Callback to process the captured image.
    """
    if image_data is None:
        print("Error: No image data received.")
        return

    try:
        # Decode the image data
        image_bytes = base64.b64decode(image_data)
        frame = Image.open(io.BytesIO(image_bytes))
        frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)

        # Debug print to verify frame capture
        print("Frame captured and processed.")

        # Process the frame
        class_name, confidence_score = drowsy_detector.process_frame(frame)

        # Display the result
        cv2.putText(frame, f"{class_name}: {confidence_score:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2_imshow(frame)
    except Exception as e:
        print(f"Error processing frame: {e}")

In [ ]:
def process_frame(self, frame):
    """
    Process the frame and make a prediction.
    """
    # Resize the frame to 224x224 pixels
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
    image_array = np.asarray(image)

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Create a batch of one image
    data = np.expand_dims(normalized_image_array, axis=0)

    # Perform prediction
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index].strip()
    confidence_score = prediction[0][index]

    # Format confidence score as a whole number
    confidence_score_whole = int(confidence_score * 100)  # Convert to percentage and remove decimals

    # Debug print to check predictions
    print(f"Prediction: {prediction}")
    print(f"Class: {class_name}, Confidence: {confidence_score_whole}%")

    # Check if the class is "drowsy"
    if class_name == "drowsy":
        self.drowsy_frame_count += 1
        print(f"Drowsy frame detected. Count: {self.drowsy_frame_count}")
    else:
        self.drowsy_frame_count = 0  # Reset the counter if not drowsy
        print("Not drowsy. Resetting counter.")

    # Debug print to check drowsy frame count
    print(f"Drowsy Frames: {self.drowsy_frame_count}")

    # Trigger alert if continuous drowsy frames exceed the threshold
    if self.drowsy_frame_count >= self.ALERT_THRESHOLD:
        print("ALERT: Drowsiness detected! Playing alert sound...")
        display(Audio(alert_sound_file, autoplay=True))

        # Display a popup message using JavaScript
        display(Javascript('alert("ALERT: Drowsiness detected! The driver is not responding.");'))

        self.drowsy_frame_count = 0  # Reset the counter after alert

    return class_name, confidence_score_whole

In [ ]:
import math

# Simulate Vehicle Class
class Vehicle:
    def __init__(self, vehicle_id, location):
        self.vehicle_id = vehicle_id
        self.location = location  # (x, y) coordinates
        self.status = "Active"  # Status of the vehicle

    def send_alert(self, message, vehicles):
        """
        Send an alert to the nearest vehicle.
        """
        nearest_vehicle = None
        min_distance = float('inf')

        # Find the nearest vehicle
        for vehicle in vehicles:
            if vehicle.vehicle_id != self.vehicle_id:  # Skip itself
                distance = self.calculate_distance(vehicle.location)
                if distance < min_distance:
                    min_distance = distance
                    nearest_vehicle = vehicle

        if nearest_vehicle:
            print(f"Vehicle {self.vehicle_id} sending alert to Vehicle {nearest_vehicle.vehicle_id}: {message}")
            nearest_vehicle.receive_alert(message)
        else:
            print("No nearby vehicles found to send the alert.")

    def receive_alert(self, message):
        """
        Receive an alert from another vehicle.
        """
        print(f"Vehicle {self.vehicle_id} received alert: {message}")
        self.status = "Assisting"  # Change status to assisting

    def calculate_distance(self, other_location):
        """
        Calculate Euclidean distance between two vehicles.
        """
        return math.sqrt((self.location[0] - other_location[0]) ** 2 + (self.location[1] - other_location[1]) ** 2)

# Simulate DrowsyDetector with V2V Communication
class DrowsyDetector:
    def __init__(self, vehicle, vehicles):
        self.vehicle = vehicle
        self.vehicles = vehicles
        self.drowsy_frame_count = 0
        self.ALERT_THRESHOLD = 5  # Number of continuous drowsy frames to trigger alert

    def process_frame(self, frame):
        """
        Simulate drowsiness detection and trigger V2V communication.
        """
        # Simulate drowsiness detection (replace with actual logic)
        is_drowsy = self.simulate_drowsiness(frame)

        if is_drowsy:
            self.drowsy_frame_count += 1
            print(f"Drowsy frame detected. Count: {self.drowsy_frame_count}")
        else:
            self.drowsy_frame_count = 0
            print("Not drowsy. Resetting counter.")

        # Trigger alert if continuous drowsy frames exceed the threshold
        if self.drowsy_frame_count >= self.ALERT_THRESHOLD:
            print("ALERT: Drowsiness detected! Sending alert to nearest vehicle...")
            self.vehicle.send_alert("Driver is not responding. Need assistance!", self.vehicles)
            self.drowsy_frame_count = 0  # Reset the counter after alert

    def simulate_drowsiness(self, frame):
        """
        Simulate drowsiness detection (replace with actual model prediction).
        """
        # Replace this with actual model prediction logic
        return True  # Simulate drowsiness for testing

# Simulate Vehicles
vehicle1 = Vehicle(vehicle_id=1, location=(0, 0))
vehicle2 = Vehicle(vehicle_id=2, location=(10, 10))
vehicle3 = Vehicle(vehicle_id=3, location=(20, 20))

# List of vehicles
vehicles = [vehicle1, vehicle2, vehicle3]

# Simulate DrowsyDetector for Vehicle 1
drowsy_detector = DrowsyDetector(vehicle1, vehicles)

# Simulate a frame (replace with actual frame processing)
frame = "simulated_frame"
drowsy_detector.process_frame(frame)

Drowsy frame detected. Count: 1


In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from PIL import Image, ImageOps
from google.colab.patches import cv2_imshow
from google.colab import output
import base64
import io
from IPython.display import display, Javascript, Audio
import math

# Load the model
model = load_model("/content/keras_model.h5", compile=False)

# Load and process labels correctly
with open("/content/labels.txt", "r") as f:
    class_names = [line.strip().split(' ', 1)[1] for line in f.readlines()]

# Path to the alert sound file (converted to .mp3)
alert_sound_file = "/content/alert.mp3"

# Simulate Vehicle Class
class Vehicle:
    def __init__(self, vehicle_id, location):
        self.vehicle_id = vehicle_id
        self.location = location  # (x, y) coordinates
        self.status = "Active"  # Status of the vehicle

    def send_alert(self, message, vehicles):
        """
        Send an alert to the nearest vehicle.
        """
        nearest_vehicle = None
        min_distance = float('inf')

        # Find the nearest vehicle
        for vehicle in vehicles:
            if vehicle.vehicle_id != self.vehicle_id:  # Skip itself
                distance = self.calculate_distance(vehicle.location)
                if distance < min_distance:
                    min_distance = distance
                    nearest_vehicle = vehicle

        if nearest_vehicle:
            print(f"Vehicle {self.vehicle_id} sending alert to Vehicle {nearest_vehicle.vehicle_id}: {message}")
            nearest_vehicle.receive_alert(message)
        else:
            print("No nearby vehicles found to send the alert.")

    def receive_alert(self, message):
        """
        Receive an alert from another vehicle.
        """
        print(f"Vehicle {self.vehicle_id} received alert: {message}")
        self.status = "Assisting"  # Change status to assisting

    def calculate_distance(self, other_location):
        """
        Calculate Euclidean distance between two vehicles.
        """
        return math.sqrt((self.location[0] - other_location[0]) ** 2 + (self.location[1] - other_location[1]) ** 2)

# Simulate Vehicles
vehicle1 = Vehicle(vehicle_id=1, location=(0, 0))
vehicle2 = Vehicle(vehicle_id=2, location=(10, 10))
vehicle3 = Vehicle(vehicle_id=3, location=(20, 20))

# List of vehicles
vehicles = [vehicle1, vehicle2, vehicle3]

# Class to maintain state
class DrowsyDetector:
    def __init__(self, vehicle, vehicles):
        self.vehicle = vehicle
        self.vehicles = vehicles
        self.drowsy_frame_count = 0
        self.ALERT_THRESHOLD = 10  # Number of continuous drowsy frames to trigger alert

    def process_frame(self, frame):
        """
        Process the frame and make a prediction.
        """
        # Resize the frame to 224x224 pixels
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
        image_array = np.asarray(image)

        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

        # Create a batch of one image
        data = np.expand_dims(normalized_image_array, axis=0)

        # Perform prediction
        prediction = model.predict(data)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        # Format confidence score as a whole number
        confidence_score_whole = int(confidence_score * 100)

        # Check if the class is "Drowsy" (index 0)
        if index == 0:  # Directly check the index instead of class name string
            self.drowsy_frame_count += 1
            print(f"Drowsy frame detected. Count: {self.drowsy_frame_count}")
        else:
            self.drowsy_frame_count = 0
            print("Not drowsy. Resetting counter.")

        # Trigger alert if continuous drowsy frames exceed the threshold
        if self.drowsy_frame_count >= self.ALERT_THRESHOLD:
            print("ALERT: Drowsiness detected! Sending alert to nearest vehicle...")
            self.vehicle.send_alert("Driver is not responding. Need assistance!", self.vehicles)
            display(Audio(alert_sound_file, autoplay=True))  # Play alert sound
            self.drowsy_frame_count = 0  # Reset the counter after alert

        return class_name, confidence_score_whole

# Create an instance of the DrowsyDetector class
drowsy_detector = DrowsyDetector(vehicle1, vehicles)

def capture_image():
    """
    Function to capture an image from the webcam via JavaScript.
    """
    js = Javascript('''
        async function takePhoto() {
            const div = document.createElement('div');
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const button = document.createElement('button');
            button.textContent = 'Stop';

            div.appendChild(video);
            div.appendChild(button);
            document.body.appendChild(div);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            return new Promise((resolve) => {
                button.onclick = () => {
                    stream.getTracks().forEach(track => track.stop());
                    div.remove();
                };

                setInterval(async () => {
                    if (!video.srcObject.active) {
                        resolve(null);
                        return;
                    }
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;
                    canvas.getContext('2d').drawImage(video, 0, 0);
                    const data = canvas.toDataURL('image/jpeg').split(',')[1];
                    google.colab.kernel.invokeFunction('notebook.captureFrameCallback', [data], {});
                }, 1000); // Capture every second
            });
        }
        takePhoto();
    ''')
    display(js)

def capture_frame_callback(image_data):
    """
    Callback to process the captured image.
    """
    if image_data is None:
        print("Error: No image data received.")
        return

    try:
        # Decode the image data
        image_bytes = base64.b64decode(image_data)
        frame = Image.open(io.BytesIO(image_bytes))
        frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)

        # Process the frame
        class_name, confidence_score_whole = drowsy_detector.process_frame(frame)

        # Add drowsiness details to the frame
        text = f"Class: {class_name}\nConfidence: {confidence_score_whole}%\nDrowsy Frames: {drowsy_detector.drowsy_frame_count}"

        # Position the text on the right side of the frame
        text_x = frame.shape[1] - 300
        text_y = 50
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.8
        font_color = (0, 255, 0)
        font_thickness = 2

        # Split the text into lines
        lines = text.split('\n')
        for i, line in enumerate(lines):
            y = text_y + i * 30
            cv2.putText(frame, line, (text_x, y), font, font_scale, font_color, font_thickness, cv2.LINE_AA)

        # Display the result
        cv2_imshow(frame)
    except Exception as e:
        print(f"Error processing frame: {e}")

# Register callback function
output.register_callback('notebook.captureFrameCallback', capture_frame_callback)

# Start capturing
capture_image()

TypeError: Error when deserializing class 'DepthwiseConv2D' using config={'name': 'expanded_conv_depthwise', 'trainable': True, 'dtype': 'float32', 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'linear', 'use_bias': False, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'bias_regularizer': None, 'activity_regularizer': None, 'bias_constraint': None, 'depth_multiplier': 1, 'depthwise_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'depthwise_regularizer': None, 'depthwise_constraint': None}.

Exception encountered: Unrecognized keyword arguments passed to DepthwiseConv2D: {'groups': 1}

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from PIL import Image, ImageOps
from google.colab.patches import cv2_imshow
from google.colab import output
import base64
import io
from IPython.display import display, Javascript, Audio
import math

# Load the model
model = load_model("/content/keras_model.h5", compile=False)

# Load and process labels correctly
with open("/content/labels.txt", "r") as f:
    class_names = [line.strip().split(' ', 1)[1] for line in f.readlines()]

# Path to the alert sound file (converted to .mp3)
alert_sound_file = "/content/alert.mp3"

# Simulate Vehicle Class
class Vehicle:
    def __init__(self, vehicle_id, location):
        self.vehicle_id = vehicle_id
        self.location = location  # (x, y) coordinates
        self.status = "Active"  # Status of the vehicle

    def send_alert(self, message, vehicles):
        """
        Send an alert to the nearest vehicle.
        """
        nearest_vehicle = None
        min_distance = float('inf')

        # Find the nearest vehicle
        for vehicle in vehicles:
            if vehicle.vehicle_id != self.vehicle_id:  # Skip itself
                distance = self.calculate_distance(vehicle.location)
                if distance < min_distance:
                    min_distance = distance
                    nearest_vehicle = vehicle

        if nearest_vehicle:
            print(f"Vehicle {self.vehicle_id} sending alert to Vehicle {nearest_vehicle.vehicle_id}: {message}")
            nearest_vehicle.receive_alert(message)
        else:
            print("No nearby vehicles found to send the alert.")

    def receive_alert(self, message):
        """
        Receive an alert from another vehicle.
        """
        print(f"Vehicle {self.vehicle_id} received alert: {message}")
        self.status = "Assisting"  # Change status to assisting

    def calculate_distance(self, other_location):
        """
        Calculate Euclidean distance between two vehicles.
        """
        return math.sqrt((self.location[0] - other_location[0]) ** 2 + (self.location[1] - other_location[1]) ** 2)

# Simulate Vehicles
vehicle1 = Vehicle(vehicle_id=1, location=(0, 0))
vehicle2 = Vehicle(vehicle_id=2, location=(10, 10))
vehicle3 = Vehicle(vehicle_id=3, location=(20, 20))

# List of vehicles
vehicles = [vehicle1, vehicle2, vehicle3]

# Class to maintain state
class DrowsyDetector:
    def __init__(self, vehicle, vehicles):
        self.vehicle = vehicle
        self.vehicles = vehicles
        self.drowsy_frame_count = 0
        self.ALERT_THRESHOLD = 5  # Number of continuous drowsy frames to trigger alert sound
        self.V2V_THRESHOLD = 10  # Number of continuous drowsy frames to trigger V2V communication

    def process_frame(self, frame):
        """
        Process the frame and make a prediction.
        """
        # Resize the frame to 224x224 pixels
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
        image_array = np.asarray(image)

        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

        # Create a batch of one image
        data = np.expand_dims(normalized_image_array, axis=0)

        # Perform prediction
        prediction = model.predict(data)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        # Format confidence score as a whole number
        confidence_score_whole = int(confidence_score * 100)

        # Check if the class is "Drowsy" (index 0)
        if index == 0:  # Directly check the index instead of class name string
            self.drowsy_frame_count += 1
            print(f"Drowsy frame detected. Count: {self.drowsy_frame_count}")
        else:
            self.drowsy_frame_count = 0
            print("Not drowsy. Resetting counter.")

        # Trigger alert sound if continuous drowsy frames exceed the alert threshold
        if self.drowsy_frame_count >= self.ALERT_THRESHOLD and self.drowsy_frame_count < self.V2V_THRESHOLD:
            print("ALERT: Drowsiness detected! Playing alert sound...")
            display(Audio(alert_sound_file, autoplay=True))  # Play alert sound

        # Trigger V2V communication if continuous drowsy frames exceed the V2V threshold
        if self.drowsy_frame_count >= self.V2V_THRESHOLD:
            print("ALERT: Drowsiness detected! Sending alert to nearest vehicle...")
            self.vehicle.send_alert("Driver is not responding. Need assistance!", self.vehicles)
            display(Javascript('alert("V2V Communication: Alert sent to nearest vehicle.");'))  # Popup for V2V
            self.drowsy_frame_count = 0  # Reset the counter after alert

        return class_name, confidence_score_whole

# Create an instance of the DrowsyDetector class
drowsy_detector = DrowsyDetector(vehicle1, vehicles)

def capture_image():
    """
    Function to capture an image from the webcam via JavaScript.
    """
    js = Javascript('''
        async function takePhoto() {
            const div = document.createElement('div');
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const button = document.createElement('button');
            button.textContent = 'Stop';

            div.appendChild(video);
            div.appendChild(button);
            document.body.appendChild(div);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            return new Promise((resolve) => {
                button.onclick = () => {
                    stream.getTracks().forEach(track => track.stop());
                    div.remove();
                };

                setInterval(async () => {
                    if (!video.srcObject.active) {
                        resolve(null);
                        return;
                    }
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;
                    canvas.getContext('2d').drawImage(video, 0, 0);
                    const data = canvas.toDataURL('image/jpeg').split(',')[1];
                    google.colab.kernel.invokeFunction('notebook.captureFrameCallback', [data], {});
                }, 1000); // Capture every second
            });
        }
        takePhoto();
    ''')
    display(js)

def capture_frame_callback(image_data):
    """
    Callback to process the captured image.
    """
    if image_data is None:
        print("Error: No image data received.")
        return

    try:
        # Decode the image data
        image_bytes = base64.b64decode(image_data)
        frame = Image.open(io.BytesIO(image_bytes))
        frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)

        # Process the frame
        class_name, confidence_score_whole = drowsy_detector.process_frame(frame)

        # Add drowsiness details to the frame
        text = f"Class: {class_name}\nConfidence: {confidence_score_whole}%\nDrowsy Frames: {drowsy_detector.drowsy_frame_count}"

        # Position the text on the right side of the frame
        text_x = frame.shape[1] - 300
        text_y = 50
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.8
        font_color = (0, 255, 0)
        font_thickness = 2

        # Split the text into lines
        lines = text.split('\n')
        for i, line in enumerate(lines):
            y = text_y + i * 30
            cv2.putText(frame, line, (text_x, y), font, font_scale, font_color, font_thickness, cv2.LINE_AA)

        # Display the result
        cv2_imshow(frame)
    except Exception as e:
        print(f"Error processing frame: {e}")

# Register callback function
output.register_callback('notebook.captureFrameCallback', capture_frame_callback)

# Start capturing
capture_image()

TypeError: Error when deserializing class 'DepthwiseConv2D' using config={'name': 'expanded_conv_depthwise', 'trainable': True, 'dtype': 'float32', 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'linear', 'use_bias': False, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'bias_regularizer': None, 'activity_regularizer': None, 'bias_constraint': None, 'depth_multiplier': 1, 'depthwise_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'depthwise_regularizer': None, 'depthwise_constraint': None}.

Exception encountered: Unrecognized keyword arguments passed to DepthwiseConv2D: {'groups': 1}

In [ ]:
import cv2
import numpy as np
from keras.models import load_model
from PIL import Image, ImageOps
from google.colab.patches import cv2_imshow
from google.colab import output
import base64
import io
from IPython.display import display, Javascript, Audio
import math
import time

# Load the model
model = load_model("/content/keras_model.h5", compile=False)

# Load and process labels correctly
with open("/content/labels.txt", "r") as f:
    class_names = [line.strip().split(' ', 1)[1] for line in f.readlines()]

# Path to the alert sound file (converted to .mp3)
alert_sound_file = "/content/alert.mp3"

# Simulate Vehicle Class
class Vehicle:
    def __init__(self, vehicle_id, location):
        self.vehicle_id = vehicle_id
        self.location = location  # (x, y) coordinates
        self.status = "Active"  # Status of the vehicle

    def send_alert(self, message, vehicles):
        """
        Send an alert to the nearest vehicle.
        """
        nearest_vehicle = None
        min_distance = float('inf')

        # Find the nearest vehicle
        for vehicle in vehicles:
            if vehicle.vehicle_id != self.vehicle_id:  # Skip itself
                distance = self.calculate_distance(vehicle.location)
                if distance < min_distance:
                    min_distance = distance
                    nearest_vehicle = vehicle

        if nearest_vehicle:
            print(f"Vehicle {self.vehicle_id} sending alert to Vehicle {nearest_vehicle.vehicle_id}: {message}")
            nearest_vehicle.receive_alert(message)
        else:
            print("No nearby vehicles found to send the alert.")

    def receive_alert(self, message):
        """
        Receive an alert from another vehicle.
        """
        print(f"Vehicle {self.vehicle_id} received alert: {message}")
        self.status = "Assisting"  # Change status to assisting

    def calculate_distance(self, other_location):
        """
        Calculate Euclidean distance between two vehicles.
        """
        return math.sqrt((self.location[0] - other_location[0]) ** 2 + (self.location[1] - other_location[1]) ** 2)

# Simulate Vehicles
vehicle1 = Vehicle(vehicle_id=1, location=(0, 0))
vehicle2 = Vehicle(vehicle_id=2, location=(10, 10))
vehicle3 = Vehicle(vehicle_id=3, location=(20, 20))

# List of vehicles
vehicles = [vehicle1, vehicle2, vehicle3]

# Class to maintain state
class DrowsyDetector:
    def __init__(self, vehicle, vehicles):
        self.vehicle = vehicle
        self.vehicles = vehicles
        self.drowsy_frame_count = 0
        self.non_drowsy_frame_count = 0
        self.longest_drowsy_streak = 0
        self.longest_non_drowsy_streak = 0
        self.total_drowsy_frames = 0
        self.total_non_drowsy_frames = 0
        self.start_time = time.time()
        self.ALERT_THRESHOLD = 5  # Number of continuous drowsy frames to trigger alert sound
        self.V2V_THRESHOLD = 10  # Number of continuous drowsy frames to trigger V2V communication

    def process_frame(self, frame):
        """
        Process the frame and make a prediction.
        """
        # Resize the frame to 224x224 pixels
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
        image_array = np.asarray(image)

        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

        # Create a batch of one image
        data = np.expand_dims(normalized_image_array, axis=0)

        # Perform prediction
        prediction = model.predict(data)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        # Format confidence score as a whole number
        confidence_score_whole = int(confidence_score * 100)

        # Check if the class is "Drowsy" (index 0)
        if index == 0:  # Directly check the index instead of class name string
            self.drowsy_frame_count += 1
            self.total_drowsy_frames += 1
            if self.drowsy_frame_count > self.longest_drowsy_streak:
                self.longest_drowsy_streak = self.drowsy_frame_count
            self.non_drowsy_frame_count = 0
            print(f"Drowsy frame detected. Count: {self.drowsy_frame_count}")
        else:
            self.non_drowsy_frame_count += 1
            self.total_non_drowsy_frames += 1
            if self.non_drowsy_frame_count > self.longest_non_drowsy_streak:
                self.longest_non_drowsy_streak = self.non_drowsy_frame_count
            self.drowsy_frame_count = 0
            print("Not drowsy. Resetting drowsy counter.")

        # Trigger alert sound if continuous drowsy frames exceed the alert threshold
        if self.drowsy_frame_count >= self.ALERT_THRESHOLD and self.drowsy_frame_count < self.V2V_THRESHOLD:
            print("ALERT: Drowsiness detected! Playing alert sound...")
            display(Audio(alert_sound_file, autoplay=True))  # Play alert sound

        # Trigger V2V communication if continuous drowsy frames exceed the V2V threshold
        if self.drowsy_frame_count >= self.V2V_THRESHOLD:
            print("ALERT: Drowsiness detected! Sending alert to nearest vehicle...")
            self.vehicle.send_alert("Driver is not responding. Need assistance!", self.vehicles)
            display(Javascript('alert("V2V Communication: Alert sent to nearest vehicle.");'))  # Popup for V2V
            self.drowsy_frame_count = 0  # Reset the counter after alert

        return class_name, confidence_score_whole

    def get_final_stats(self):
        """
        Get final statistics when the webcam is stopped.
        """
        end_time = time.time()
        total_time = end_time - self.start_time

        stats = {
            "total_drowsy_frames": self.total_drowsy_frames,
            "total_non_drowsy_frames": self.total_non_drowsy_frames,
            "longest_drowsy_streak": self.longest_drowsy_streak,
            "longest_non_drowsy_streak": self.longest_non_drowsy_streak,
            "total_time": total_time
        }

        return stats

# Create an instance of the DrowsyDetector class
drowsy_detector = DrowsyDetector(vehicle1, vehicles)

def capture_image():
    """
    Function to capture an image from the webcam via JavaScript.
    """
    js = Javascript('''
        async function takePhoto() {
            const div = document.createElement('div');
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const button = document.createElement('button');
            button.textContent = 'Stop';

            div.appendChild(video);
            div.appendChild(button);
            document.body.appendChild(div);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            return new Promise((resolve) => {
                button.onclick = () => {
                    stream.getTracks().forEach(track => track.stop());
                    div.remove();
                    resolve(null);
                };

                setInterval(async () => {
                    if (!video.srcObject.active) {
                        resolve(null);
                        return;
                    }
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;
                    canvas.getContext('2d').drawImage(video, 0, 0);
                    const data = canvas.toDataURL('image/jpeg').split(',')[1];
                    google.colab.kernel.invokeFunction('notebook.captureFrameCallback', [data], {});
                }, 1000); // Capture every second
            });
        }
        takePhoto();
    ''')
    display(js)

def capture_frame_callback(image_data):
    """
    Callback to process the captured image.
    """
    if image_data is None:
        print("Stopping webcam...")
        stats = drowsy_detector.get_final_stats()
        print("\nFinal Statistics:")
        print(f"Total Drowsy Frames: {stats['total_drowsy_frames']}")
        print(f"Total Non-Drowsy Frames: {stats['total_non_drowsy_frames']}")
        print(f"Longest Drowsy Streak: {stats['longest_drowsy_streak']} frames")
        print(f"Longest Non-Drowsy Streak: {stats['longest_non_drowsy_streak']} frames")
        print(f"Total Time: {stats['total_time']:.2f} seconds")

        # Create a blank frame to display the final statistics
        frame = np.zeros((500, 800, 3), dtype=np.uint8)  # Black background

        # Add statistics to the frame
        text = [
            "Final Statistics:",
            "-----------------",
            f"Total Drowsy Frames: {stats['total_drowsy_frames']}",
            f"Total Non-Drowsy Frames: {stats['total_non_drowsy_frames']}",
            f"Longest Drowsy Streak: {stats['longest_drowsy_streak']} frames",
            f"Longest Non-Drowsy Streak: {stats['longest_non_drowsy_streak']} frames",
            f"Total Time: {stats['total_time']:.2f} seconds"
        ]

        # Position the text on the left side of the frame
        text_x = 50
        text_y = 50
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.8
        font_color = (0, 255, 0)  # Green color
        font_thickness = 2

        # Add each line of text to the frame
        for i, line in enumerate(text):
            y = text_y + i * 30
            cv2.putText(frame, line, (text_x, y), font, font_scale, font_color, font_thickness, cv2.LINE_AA)

        # Display the final frame with statistics
        cv2_imshow(frame)
        return

    try:
        # Decode the image data
        image_bytes = base64.b64decode(image_data)
        frame = Image.open(io.BytesIO(image_bytes))
        frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)

        # Process the frame
        class_name, confidence_score_whole = drowsy_detector.process_frame(frame)

        # Add drowsiness details to the frame
        text = f"Class: {class_name}\nConfidence: {confidence_score_whole}%\nDrowsy Frames: {drowsy_detector.drowsy_frame_count}\nNon-Drowsy Frames: {drowsy_detector.non_drowsy_frame_count}"

        # Position the text on the right side of the frame
        text_x = frame.shape[1] - 300
        text_y = 50
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.8
        font_color = (0, 255, 0)
        font_thickness = 2

        # Split the text into lines
        lines = text.split('\n')
        for i, line in enumerate(lines):
            y = text_y + i * 30
            cv2.putText(frame, line, (text_x, y), font, font_scale, font_color, font_thickness, cv2.LINE_AA)

        # Display the result
        cv2_imshow(frame)
    except Exception as e:
        print(f"Error processing frame: {e}")

# Register callback function
output.register_callback('notebook.captureFrameCallback', capture_frame_callback)

# Start capturing
capture_image()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
from google.colab import files

# Upload the V2V Communication files
uploaded = files.upload()

# List the uploaded files
for filename in uploaded.keys():
    print(f"Uploaded {filename}")

Saving New folder.zip to New folder.zip
Uploaded New folder.zip


In [ ]:
import subprocess

# Start the V2V server
v2v_server_process = subprocess.Popen(["python", "v2v_server.py"])

# Start the Traffic Police server
traffic_police_server_process = subprocess.Popen(["python", "traffic_police_server.py"])

In [ ]:
!pip install python-socketio
!pip install flask-socketio
!pip install pyttsx3
!pip install speechrecognition
!pip install eventlet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.3/363.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 23.7 MB/s eta 0:00:00


In [ ]:
import socketio
import flask_socketio
import pyttsx3
import speech_recognition as sr
import eventlet

In [ ]:
# Install required libraries
!pip install python-socketio
!pip install flask-socketio
!pip install pyttsx3
!pip install speechrecognition
!pip install eventlet

# Import libraries
import cv2
import numpy as np
from keras.models import load_model
from PIL import Image, ImageOps
from google.colab.patches import cv2_imshow
from google.colab import output
import base64
import io
from IPython.display import display, Javascript, Audio
import math
import time
import threading
import socketio
import queue
import pyttsx3
import os

# Load the drowsy detection model
model = load_model("/content/keras_model.h5", compile=False)

# Load and process labels correctly
with open("/content/labels.txt", "r") as f:
    class_names = [line.strip().split(' ', 1)[1] for line in f.readlines()]

# Path to the alert sound file (converted to .mp3)
alert_sound_file = "/content/alert.mp3"

# Initialize the V2V client
sio = socketio.Client()

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Create a queue for text-to-speech requests
speech_queue = queue.Queue()

# Function to process text-to-speech requests
def speech_worker():
    while True:
        text = speech_queue.get()
        if text is None:
            break
        engine.say(text)
        engine.runAndWait()
        speech_queue.task_done()

# Start the speech worker thread
speech_thread = threading.Thread(target=speech_worker)
speech_thread.daemon = True
speech_thread.start()

# Function to add text to the speech queue
def speak(text):
    speech_queue.put(text)

# Event for when the client connects to the server
@sio.event
def connect():
    print("Connected to the V2V server")
    speak("Connected to the V2V server")

# Event for when the client disconnects from the server
@sio.event
def disconnect():
    print("Disconnected from the V2V server")
    speak("Disconnected from the V2V server")

# Event for receiving a response from the server
@sio.event
def v2v_response(data):
    print(f"Received response: {data}")
    speak(f"Received response: {data}")

# Connect to the V2V server
try:
    sio.connect('http://127.0.0.1:5000')
except Exception as e:
    print(f"Failed to connect to the V2V server: {e}")
    speak("Failed to connect to the V2V server")

# Class to maintain state for drowsy detection
class DrowsyDetector:
    def __init__(self):
        self.drowsy_frame_count = 0
        self.non_drowsy_frame_count = 0
        self.longest_drowsy_streak = 0
        self.longest_non_drowsy_streak = 0
        self.total_drowsy_frames = 0
        self.total_non_drowsy_frames = 0
        self.start_time = time.time()
        self.ALERT_THRESHOLD = 5  # Number of continuous drowsy frames to trigger alert sound
        self.V2V_THRESHOLD = 10  # Number of continuous drowsy frames to trigger V2V communication

    def process_frame(self, frame):
        """
        Process the frame and make a prediction.
        """
        # Resize the frame to 224x224 pixels
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
        image_array = np.asarray(image)

        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

        # Create a batch of one image
        data = np.expand_dims(normalized_image_array, axis=0)

        # Perform prediction
        prediction = model.predict(data)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        # Format confidence score as a whole number
        confidence_score_whole = int(confidence_score * 100)

        # Check if the class is "Drowsy" (index 0)
        if index == 0:  # Directly check the index instead of class name string
            self.drowsy_frame_count += 1
            self.total_drowsy_frames += 1
            if self.drowsy_frame_count > self.longest_drowsy_streak:
                self.longest_drowsy_streak = self.drowsy_frame_count
            self.non_drowsy_frame_count = 0
            print(f"Drowsy frame detected. Count: {self.drowsy_frame_count}")
        else:
            self.non_drowsy_frame_count += 1
            self.total_non_drowsy_frames += 1
            if self.non_drowsy_frame_count > self.longest_non_drowsy_streak:
                self.longest_non_drowsy_streak = self.non_drowsy_frame_count
            self.drowsy_frame_count = 0
            print("Not drowsy. Resetting drowsy counter.")

        # Trigger alert sound if continuous drowsy frames exceed the alert threshold
        if self.drowsy_frame_count >= self.ALERT_THRESHOLD and self.drowsy_frame_count < self.V2V_THRESHOLD:
            print("ALERT: Drowsiness detected! Playing alert sound...")
            display(Audio(alert_sound_file, autoplay=True))  # Play alert sound

        # Trigger V2V communication if continuous drowsy frames exceed the V2V threshold
        if self.drowsy_frame_count >= self.V2V_THRESHOLD:
            print("ALERT: Drowsiness detected! Sending alert to V2V server...")
            sio.emit('v2v_message', {'message': "Driver is not responding. Need assistance!", 'type': 'emergency'})
            self.drowsy_frame_count = 0  # Reset the counter after alert

        return class_name, confidence_score_whole

    def get_final_stats(self):
        """
        Get final statistics when the webcam is stopped.
        """
        end_time = time.time()
        total_time = end_time - self.start_time

        stats = {
            "total_drowsy_frames": self.total_drowsy_frames,
            "total_non_drowsy_frames": self.total_non_drowsy_frames,
            "longest_drowsy_streak": self.longest_drowsy_streak,
            "longest_non_drowsy_streak": self.longest_non_drowsy_streak,
            "total_time": total_time
        }

        return stats

# Create an instance of the DrowsyDetector class
drowsy_detector = DrowsyDetector()

# Function to capture an image from the webcam via JavaScript
def capture_image():
    js = Javascript('''
        async function takePhoto() {
            const div = document.createElement('div');
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const button = document.createElement('button');
            button.textContent = 'Stop';

            div.appendChild(video);
            div.appendChild(button);
            document.body.appendChild(div);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            return new Promise((resolve) => {
                button.onclick = () => {
                    stream.getTracks().forEach(track => track.stop());
                    div.remove();
                    resolve(null);
                };

                setInterval(async () => {
                    if (!video.srcObject.active) {
                        resolve(null);
                        return;
                    }
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;
                    canvas.getContext('2d').drawImage(video, 0, 0);
                    const data = canvas.toDataURL('image/jpeg').split(',')[1];
                    google.colab.kernel.invokeFunction('notebook.captureFrameCallback', [data], {});
                }, 1000); // Capture every second
            });
        }
        takePhoto();
    ''')
    display(js)

# Callback to process the captured image
def capture_frame_callback(image_data):
    if image_data is None:
        print("Stopping webcam...")
        stats = drowsy_detector.get_final_stats()
        print("\nFinal Statistics:")
        print(f"Total Drowsy Frames: {stats['total_drowsy_frames']}")
        print(f"Total Non-Drowsy Frames: {stats['total_non_drowsy_frames']}")
        print(f"Longest Drowsy Streak: {stats['longest_drowsy_streak']} frames")
        print(f"Longest Non-Drowsy Streak: {stats['longest_non_drowsy_streak']} frames")
        print(f"Total Time: {stats['total_time']:.2f} seconds")
        return

    try:
        # Decode the image data
        image_bytes = base64.b64decode(image_data)
        frame = Image.open(io.BytesIO(image_bytes))
        frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)

        # Process the frame
        class_name, confidence_score_whole = drowsy_detector.process_frame(frame)

        # Add drowsiness details to the frame
        text = f"Class: {class_name}\nConfidence: {confidence_score_whole}%\nDrowsy Frames: {drowsy_detector.drowsy_frame_count}\nNon-Drowsy Frames: {drowsy_detector.non_drowsy_frame_count}"

        # Position the text on the right side of the frame
        text_x = frame.shape[1] - 300
        text_y = 50
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.8
        font_color = (0, 255, 0)
        font_thickness = 2

        # Split the text into lines
        lines = text.split('\n')
        for i, line in enumerate(lines):
            y = text_y + i * 30
            cv2.putText(frame, line, (text_x, y), font, font_scale, font_color, font_thickness, cv2.LINE_AA)

        # Display the result
        cv2_imshow(frame)
    except Exception as e:
        print(f"Error processing frame: {e}")

# Register callback function
output.register_callback('notebook.captureFrameCallback', capture_frame_callback)

# Start capturing
capture_image()

RuntimeError: This means you probably do not have eSpeak or eSpeak-ng installed!

In [ ]:
!sudo apt-get update
!sudo apt-get install espeak

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,285 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,648 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jamm

In [ ]:
!espeak --version

eSpeak text-to-speech: 1.48.15  16.Apr.15  Data at: /usr/lib/x86_64-linux-gnu/espeak-data


In [ ]:
import pyttsx3

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Test the engine
engine.say("Hello, this is a test.")
engine.runAndWait()

In [ ]:
import subprocess
import time

# Start the V2V server
v2v_server_process = subprocess.Popen(["python", "v2v_server.py"])

# Start the Traffic Police server
traffic_police_server_process = subprocess.Popen(["python", "traffic_police_server.py"])

# Wait for the servers to start
time.sleep(5)

print("V2V Server and Traffic Police Server are running in the background.")

V2V Server and Traffic Police Server are running in the background.


In [ ]:
# Install required libraries
!pip install python-socketio
!pip install flask-socketio
!pip install pyttsx3
!pip install speechrecognition
!pip install eventlet

# Install eSpeak
!sudo apt-get update
!sudo apt-get install espeak

# Import libraries
import cv2
import numpy as np
from keras.models import load_model
from PIL import Image, ImageOps
from google.colab.patches import cv2_imshow
from google.colab import output
import base64
import io
from IPython.display import display, Javascript, Audio
import math
import time
import threading
import socketio
import queue
import pyttsx3
import os

# Load the drowsy detection model
model = load_model("/content/keras_model.h5", compile=False)

# Load and process labels correctly
with open("/content/labels.txt", "r") as f:
    class_names = [line.strip().split(' ', 1)[1] for line in f.readlines()]

# Path to the alert sound file (converted to .mp3)
alert_sound_file = "/content/alert.mp3"

# Initialize the V2V client
sio = socketio.Client()

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Create a queue for text-to-speech requests
speech_queue = queue.Queue()

# Function to process text-to-speech requests
def speech_worker():
    while True:
        text = speech_queue.get()
        if text is None:
            break
        engine.say(text)
        engine.runAndWait()
        speech_queue.task_done()

# Start the speech worker thread
speech_thread = threading.Thread(target=speech_worker)
speech_thread.daemon = True
speech_thread.start()

# Function to add text to the speech queue
def speak(text):
    speech_queue.put(text)

# Event for when the client connects to the server
@sio.event
def connect():
    print("Connected to the V2V server")
    speak("Connected to the V2V server")

# Event for when the client disconnects from the server
@sio.event
def disconnect():
    print("Disconnected from the V2V server")
    speak("Disconnected from the V2V server")

# Event for receiving a response from the server
@sio.event
def v2v_response(data):
    print(f"Received response: {data}")
    speak(f"Received response: {data}")

# Connect to the V2V server
try:
    sio.connect('http://127.0.0.1:5000')
except Exception as e:
    print(f"Failed to connect to the V2V server: {e}")
    speak("Failed to connect to the V2V server")

# Class to maintain state for drowsy detection
class DrowsyDetector:
    def __init__(self):
        self.drowsy_frame_count = 0
        self.non_drowsy_frame_count = 0
        self.longest_drowsy_streak = 0
        self.longest_non_drowsy_streak = 0
        self.total_drowsy_frames = 0
        self.total_non_drowsy_frames = 0
        self.start_time = time.time()
        self.ALERT_THRESHOLD = 5  # Number of continuous drowsy frames to trigger alert sound
        self.V2V_THRESHOLD = 10  # Number of continuous drowsy frames to trigger V2V communication

    def process_frame(self, frame):
        """
        Process the frame and make a prediction.
        """
        # Resize the frame to 224x224 pixels
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
        image_array = np.asarray(image)

        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

        # Create a batch of one image
        data = np.expand_dims(normalized_image_array, axis=0)

        # Perform prediction
        prediction = model.predict(data)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        # Format confidence score as a whole number
        confidence_score_whole = int(confidence_score * 100)

        # Check if the class is "Drowsy" (index 0)
        if index == 0:  # Directly check the index instead of class name string
            self.drowsy_frame_count += 1
            self.total_drowsy_frames += 1
            if self.drowsy_frame_count > self.longest_drowsy_streak:
                self.longest_drowsy_streak = self.drowsy_frame_count
            self.non_drowsy_frame_count = 0
            print(f"Drowsy frame detected. Count: {self.drowsy_frame_count}")
        else:
            self.non_drowsy_frame_count += 1
            self.total_non_drowsy_frames += 1
            if self.non_drowsy_frame_count > self.longest_non_drowsy_streak:
                self.longest_non_drowsy_streak = self.non_drowsy_frame_count
            self.drowsy_frame_count = 0
            print("Not drowsy. Resetting drowsy counter.")

        # Trigger alert sound if continuous drowsy frames exceed the alert threshold
        if self.drowsy_frame_count >= self.ALERT_THRESHOLD and self.drowsy_frame_count < self.V2V_THRESHOLD:
            print("ALERT: Drowsiness detected! Playing alert sound...")
            display(Audio(alert_sound_file, autoplay=True))  # Play alert sound

        # Trigger V2V communication if continuous drowsy frames exceed the V2V threshold
        if self.drowsy_frame_count >= self.V2V_THRESHOLD:
            print("ALERT: Drowsiness detected! Sending alert to V2V server...")
            sio.emit('v2v_message', {'message': "Driver is not responding. Need assistance!", 'type': 'emergency'})
            self.drowsy_frame_count = 0  # Reset the counter after alert

        return class_name, confidence_score_whole

    def get_final_stats(self):
        """
        Get final statistics when the webcam is stopped.
        """
        end_time = time.time()
        total_time = end_time - self.start_time

        stats = {
            "total_drowsy_frames": self.total_drowsy_frames,
            "total_non_drowsy_frames": self.total_non_drowsy_frames,
            "longest_drowsy_streak": self.longest_drowsy_streak,
            "longest_non_drowsy_streak": self.longest_non_drowsy_streak,
            "total_time": total_time
        }

        return stats

# Create an instance of the DrowsyDetector class
drowsy_detector = DrowsyDetector()

# Function to capture an image from the webcam via JavaScript
def capture_image():
    js = Javascript('''
        async function takePhoto() {
            const div = document.createElement('div');
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const button = document.createElement('button');
            button.textContent = 'Stop';

            div.appendChild(video);
            div.appendChild(button);
            document.body.appendChild(div);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            return new Promise((resolve) => {
                button.onclick = () => {
                    stream.getTracks().forEach(track => track.stop());
                    div.remove();
                    resolve(null);
                };

                setInterval(async () => {
                    if (!video.srcObject.active) {
                        resolve(null);
                        return;
                    }
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;
                    canvas.getContext('2d').drawImage(video, 0, 0);
                    const data = canvas.toDataURL('image/jpeg').split(',')[1];
                    google.colab.kernel.invokeFunction('notebook.captureFrameCallback', [data], {});
                }, 1000); // Capture every second
            });
        }
        takePhoto();
    ''')
    display(js)

# Callback to process the captured image
def capture_frame_callback(image_data):
    if image_data is None:
        print("Stopping webcam...")
        stats = drowsy_detector.get_final_stats()
        print("\nFinal Statistics:")
        print(f"Total Drowsy Frames: {stats['total_drowsy_frames']}")
        print(f"Total Non-Drowsy Frames: {stats['total_non_drowsy_frames']}")
        print(f"Longest Drowsy Streak: {stats['longest_drowsy_streak']} frames")
        print(f"Longest Non-Drowsy Streak: {stats['longest_non_drowsy_streak']} frames")
        print(f"Total Time: {stats['total_time']:.2f} seconds")
        return

    try:
        # Decode the image data
        image_bytes = base64.b64decode(image_data)
        frame = Image.open(io.BytesIO(image_bytes))
        frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)

        # Process the frame
        class_name, confidence_score_whole = drowsy_detector.process_frame(frame)

        # Add drowsiness details to the frame
        text = f"Class: {class_name}\nConfidence: {confidence_score_whole}%\nDrowsy Frames: {drowsy_detector.drowsy_frame_count}\nNon-Drowsy Frames: {drowsy_detector.non_drowsy_frame_count}"

        # Position the text on the right side of the frame
        text_x = frame.shape[1] - 300
        text_y = 50
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.8
        font_color = (0, 255, 0)
        font_thickness = 2

        # Split the text into lines
        lines = text.split('\n')
        for i, line in enumerate(lines):
            y = text_y + i * 30
            cv2.putText(frame, line, (text_x, y), font, font_scale, font_color, font_thickness, cv2.LINE_AA)

        # Display the result
        cv2_imshow(frame)
    except Exception as e:
        print(f"Error processing frame: {e}")

# Register callback function
output.register_callback('notebook.captureFrameCallback', capture_frame_callback)

# Start capturing
capture_image()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 6,555 B in 1s (5,275 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... D

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/keras_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
import subprocess
import time

# Start the V2V server
v2v_server_process = subprocess.Popen(["python", "v2v_server.py"])

# Start the Traffic Police server
traffic_police_server_process = subprocess.Popen(["python", "traffic_police_server.py"])

# Wait for the servers to start
time.sleep(5)

print("V2V Server and Traffic Police Server are running in the background.")

V2V Server and Traffic Police Server are running in the background.


In [ ]:
# Install required libraries
!pip install python-socketio
!pip install flask-socketio
!pip install pyttsx3
!pip install speechrecognition
!pip install eventlet

# Install eSpeak
!sudo apt-get update
!sudo apt-get install espeak

# Import libraries
import cv2
import numpy as np
from keras.models import load_model
from PIL import Image, ImageOps
from google.colab.patches import cv2_imshow
from google.colab import output
import base64
import io
from IPython.display import display, Javascript, Audio
import math
import time
import threading
import socketio
import queue
import pyttsx3
import os

# Load the drowsy detection model
model = load_model("/content/keras_model.h5", compile=False)

# Load and process labels correctly
with open("/content/labels.txt", "r") as f:
    class_names = [line.strip().split(' ', 1)[1] for line in f.readlines()]

# Path to the alert sound file (converted to .mp3)
alert_sound_file = "/content/alert.mp3"

# Initialize the V2V client
sio = socketio.Client()

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Create a queue for text-to-speech requests
speech_queue = queue.Queue()

# Function to process text-to-speech requests
def speech_worker():
    while True:
        text = speech_queue.get()
        if text is None:
            break
        engine.say(text)
        engine.runAndWait()
        speech_queue.task_done()

# Start the speech worker thread
speech_thread = threading.Thread(target=speech_worker)
speech_thread.daemon = True
speech_thread.start()

# Function to add text to the speech queue
def speak(text):
    speech_queue.put(text)

# Event for when the client connects to the server
@sio.event
def connect():
    print("Connected to the V2V server")
    speak("Connected to the V2V server")

# Event for when the client disconnects from the server
@sio.event
def disconnect():
    print("Disconnected from the V2V server")
    speak("Disconnected from the V2V server")

# Event for receiving a response from the server
@sio.event
def v2v_response(data):
    print(f"Received response: {data}")
    speak(f"Received response: {data}")

# Connect to the V2V server
try:
    sio.connect('http://127.0.0.1:5000')
except Exception as e:
    print(f"Failed to connect to the V2V server: {e}")
    speak("Failed to connect to the V2V server")

# Class to maintain state for drowsy detection
class DrowsyDetector:
    def __init__(self):
        self.drowsy_frame_count = 0
        self.non_drowsy_frame_count = 0
        self.longest_drowsy_streak = 0
        self.longest_non_drowsy_streak = 0
        self.total_drowsy_frames = 0
        self.total_non_drowsy_frames = 0
        self.start_time = time.time()
        self.ALERT_THRESHOLD = 5  # Number of continuous drowsy frames to trigger alert sound
        self.V2V_THRESHOLD = 10  # Number of continuous drowsy frames to trigger V2V communication

    def process_frame(self, frame):
        """
        Process the frame and make a prediction.
        """
        # Resize the frame to 224x224 pixels
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
        image_array = np.asarray(image)

        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

        # Create a batch of one image
        data = np.expand_dims(normalized_image_array, axis=0)

        # Perform prediction
        prediction = model.predict(data)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        # Format confidence score as a whole number
        confidence_score_whole = int(confidence_score * 100)

        # Check if the class is "Drowsy" (index 0)
        if index == 0:  # Directly check the index instead of class name string
            self.drowsy_frame_count += 1
            self.total_drowsy_frames += 1
            if self.drowsy_frame_count > self.longest_drowsy_streak:
                self.longest_drowsy_streak = self.drowsy_frame_count
            self.non_drowsy_frame_count = 0
            print(f"Drowsy frame detected. Count: {self.drowsy_frame_count}")

            # Trigger alert sound if continuous drowsy frames exceed the alert threshold
            if self.drowsy_frame_count == self.ALERT_THRESHOLD:
                print("ALERT: Drowsiness detected! Playing alert sound...")
                display(Audio(alert_sound_file, autoplay=True))  # Play alert sound

            # Trigger V2V communication if continuous drowsy frames exceed the V2V threshold
            if self.drowsy_frame_count == self.V2V_THRESHOLD:
                print("ALERT: Drowsiness detected! Sending alert to V2V server and Traffic Police server...")
                sio.emit('v2v_message', {'message': "Driver is not responding. Need assistance!", 'type': 'emergency'})
                sio.emit('traffic_police_message', {'message': "Driver is not responding. Need assistance!", 'type': 'emergency'})
                self.drowsy_frame_count = 0  # Reset the counter after alert
        else:
            self.non_drowsy_frame_count += 1
            self.total_non_drowsy_frames += 1
            if self.non_drowsy_frame_count > self.longest_non_drowsy_streak:
                self.longest_non_drowsy_streak = self.non_drowsy_frame_count
            self.drowsy_frame_count = 0
            print("Not drowsy. Resetting drowsy counter.")

        return class_name, confidence_score_whole

    def get_final_stats(self):
        """
        Get final statistics when the webcam is stopped.
        """
        end_time = time.time()
        total_time = end_time - self.start_time

        stats = {
            "total_drowsy_frames": self.total_drowsy_frames,
            "total_non_drowsy_frames": self.total_non_drowsy_frames,
            "longest_drowsy_streak": self.longest_drowsy_streak,
            "longest_non_drowsy_streak": self.longest_non_drowsy_streak,
            "total_time": total_time
        }

        return stats

# Create an instance of the DrowsyDetector class
drowsy_detector = DrowsyDetector()

# Function to capture an image from the webcam via JavaScript
def capture_image():
    js = Javascript('''
        async function takePhoto() {
            const div = document.createElement('div');
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const button = document.createElement('button');
            button.textContent = 'Stop';

            div.appendChild(video);
            div.appendChild(button);
            document.body.appendChild(div);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            return new Promise((resolve) => {
                button.onclick = () => {
                    stream.getTracks().forEach(track => track.stop());
                    div.remove();
                    resolve(null);
                };

                setInterval(async () => {
                    if (!video.srcObject.active) {
                        resolve(null);
                        return;
                    }
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;
                    canvas.getContext('2d').drawImage(video, 0, 0);
                    const data = canvas.toDataURL('image/jpeg').split(',')[1];
                    google.colab.kernel.invokeFunction('notebook.captureFrameCallback', [data], {});
                }, 1000); // Capture every second
            });
        }
        takePhoto();
    ''')
    display(js)

# Callback to process the captured image
def capture_frame_callback(image_data):
    if image_data is None:
        print("Stopping webcam...")
        stats = drowsy_detector.get_final_stats()
        print("\nFinal Statistics:")
        print(f"Total Drowsy Frames: {stats['total_drowsy_frames']}")
        print(f"Total Non-Drowsy Frames: {stats['total_non_drowsy_frames']}")
        print(f"Longest Drowsy Streak: {stats['longest_drowsy_streak']} frames")
        print(f"Longest Non-Drowsy Streak: {stats['longest_non_drowsy_streak']} frames")
        print(f"Total Time: {stats['total_time']:.2f} seconds")
        return

    try:
        # Decode the image data
        image_bytes = base64.b64decode(image_data)
        frame = Image.open(io.BytesIO(image_bytes))
        frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)

        # Process the frame
        class_name, confidence_score_whole = drowsy_detector.process_frame(frame)

        # Add drowsiness details to the frame
        text = f"Class: {class_name}\nConfidence: {confidence_score_whole}%\nDrowsy Frames: {drowsy_detector.drowsy_frame_count}\nNon-Drowsy Frames: {drowsy_detector.non_drowsy_frame_count}"

        # Position the text on the right side of the frame
        text_x = frame.shape[1] - 300
        text_y = 50
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.8
        font_color = (0, 255, 0)
        font_thickness = 2

        # Split the text into lines
        lines = text.split('\n')
        for i, line in enumerate(lines):
            y = text_y + i * 30
            cv2.putText(frame, line, (text_x, y), font, font_scale, font_color, font_thickness, cv2.LINE_AA)

        # Display the result
        cv2_imshow(frame)
    except Exception as e:
        print(f"Error processing frame: {e}")

# Register callback function
output.register_callback('notebook.captureFrameCallback', capture_frame_callback)

# Start capturing
capture_image()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 129 kB in 2s (71.8 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Don

<IPython.core.display.Javascript object>

In [ ]:

# Import libraries
import cv2
import numpy as np
from keras.models import load_model
from PIL import Image, ImageOps
from google.colab.patches import cv2_imshow
from google.colab import output
import base64
import io
from IPython.display import display, Javascript, Audio
import math
import time
import threading
import socketio
import queue
import pyttsx3
import os

# Load the drowsy detection model
model = load_model("/content/keras_model.h5", compile=False)

# Load and process labels correctly
with open("/content/labels.txt", "r") as f:
    class_names = [line.strip().split(' ', 1)[1] for line in f.readlines()]

# Path to the alert sound file (converted to .mp3)
alert_sound_file = "/content/alert.mp3"

# Initialize the V2V client
sio = socketio.Client()

# Initialize the text-to-speech engine
engine = pyttsx3.init()

# Create a queue for text-to-speech requests
speech_queue = queue.Queue()

# Function to process text-to-speech requests
def speech_worker():
    while True:
        text = speech_queue.get()
        if text is None:
            break
        engine.say(text)
        engine.runAndWait()
        speech_queue.task_done()

# Start the speech worker thread
speech_thread = threading.Thread(target=speech_worker)
speech_thread.daemon = True
speech_thread.start()

# Function to add text to the speech queue
def speak(text):
    speech_queue.put(text)

# Event for when the client connects to the server
@sio.event
def connect():
    print("Connected to the V2V server")
    speak("Connected to the V2V server")

# Event for when the client disconnects from the server
@sio.event
def disconnect():
    print("Disconnected from the V2V server")
    speak("Disconnected from the V2V server")

# Event for receiving a response from the server
@sio.event
def v2v_response(data):
    print(f"Received response: {data}")
    speak(f"Received response: {data}")

# Connect to the V2V server
try:
    sio.connect('http://127.0.0.1:5000')
except Exception as e:
    print(f"Failed to connect to the V2V server: {e}")
    speak("Failed to connect to the V2V server")

# Class to maintain state for drowsy detection
class DrowsyDetector:
    def __init__(self):
        self.drowsy_frame_count = 0
        self.non_drowsy_frame_count = 0
        self.longest_drowsy_streak = 0
        self.longest_non_drowsy_streak = 0
        self.total_drowsy_frames = 0
        self.total_non_drowsy_frames = 0
        self.start_time = time.time()
        self.ALERT_THRESHOLD = 5  # Number of continuous drowsy frames to trigger alert sound
        self.V2V_THRESHOLD = 10  # Number of continuous drowsy frames to trigger V2V communication

    def process_frame(self, frame):
        """
        Process the frame and make a prediction.
        """
        # Resize the frame to 224x224 pixels
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
        image_array = np.asarray(image)

        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

        # Create a batch of one image
        data = np.expand_dims(normalized_image_array, axis=0)

        # Perform prediction
        prediction = model.predict(data)
        index = np.argmax(prediction)
        class_name = class_names[index]
        confidence_score = prediction[0][index]

        # Format confidence score as a whole number
        confidence_score_whole = int(confidence_score * 100)

        # Check if the class is "Drowsy" (index 0)
        if index == 0:  # Directly check the index instead of class name string
            self.drowsy_frame_count += 1
            self.total_drowsy_frames += 1
            if self.drowsy_frame_count > self.longest_drowsy_streak:
                self.longest_drowsy_streak = self.drowsy_frame_count
            self.non_drowsy_frame_count = 0
            print(f"Drowsy frame detected. Count: {self.drowsy_frame_count}")

            # Trigger alert sound if continuous drowsy frames exceed the alert threshold
            if self.drowsy_frame_count == self.ALERT_THRESHOLD:
                print("ALERT: Drowsiness detected! Playing alert sound...")
                display(Audio(alert_sound_file, autoplay=True))  # Play alert sound

            # Trigger V2V communication if continuous drowsy frames exceed the V2V threshold
            if self.drowsy_frame_count == self.V2V_THRESHOLD:
                print("ALERT: Drowsiness detected! Sending alert to V2V server and Traffic Police server...")
                sio.emit('v2v_message', {'message': "Driver is not responding. Need assistance!", 'type': 'emergency'})
                sio.emit('traffic_police_message', {'message': "Driver is not responding. Need assistance!", 'type': 'emergency'})
                self.drowsy_frame_count = 0  # Reset the counter after alert
        else:
            self.non_drowsy_frame_count += 1
            self.total_non_drowsy_frames += 1
            if self.non_drowsy_frame_count > self.longest_non_drowsy_streak:
                self.longest_non_drowsy_streak = self.non_drowsy_frame_count
            self.drowsy_frame_count = 0
            print("Not drowsy. Resetting drowsy counter.")

        return class_name, confidence_score_whole

    def get_final_stats(self):
        """
        Get final statistics when the webcam is stopped.
        """
        end_time = time.time()
        total_time = end_time - self.start_time

        stats = {
            "total_drowsy_frames": self.total_drowsy_frames,
            "total_non_drowsy_frames": self.total_non_drowsy_frames,
            "longest_drowsy_streak": self.longest_drowsy_streak,
            "longest_non_drowsy_streak": self.longest_non_drowsy_streak,
            "total_time": total_time
        }

        return stats

# Create an instance of the DrowsyDetector class
drowsy_detector = DrowsyDetector()

# Function to capture an image from the webcam via JavaScript
def capture_image():
    js = Javascript('''
        async function takePhoto() {
            const div = document.createElement('div');
            const video = document.createElement('video');
            const canvas = document.createElement('canvas');
            const button = document.createElement('button');
            button.textContent = 'Stop';

            div.appendChild(video);
            div.appendChild(button);
            document.body.appendChild(div);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();

            return new Promise((resolve) => {
                button.onclick = () => {
                    stream.getTracks().forEach(track => track.stop());
                    div.remove();
                    resolve(null);
                };

                setInterval(async () => {
                    if (!video.srcObject.active) {
                        resolve(null);
                        return;
                    }
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;
                    canvas.getContext('2d').drawImage(video, 0, 0);
                    const data = canvas.toDataURL('image/jpeg').split(',')[1];
                    google.colab.kernel.invokeFunction('notebook.captureFrameCallback', [data], {});
                }, 1000); // Capture every second
            });
        }
        takePhoto();
    ''')
    display(js)

# Callback to process the captured image
def capture_frame_callback(image_data):
    if image_data is None:
        print("Stopping webcam...")
        stats = drowsy_detector.get_final_stats()
        print("\nFinal Statistics:")
        print(f"Total Drowsy Frames: {stats['total_drowsy_frames']}")
        print(f"Total Non-Drowsy Frames: {stats['total_non_drowsy_frames']}")
        print(f"Longest Drowsy Streak: {stats['longest_drowsy_streak']} frames")
        print(f"Longest Non-Drowsy Streak: {stats['longest_non_drowsy_streak']} frames")
        print(f"Total Time: {stats['total_time']:.2f} seconds")
        return

    try:
        # Decode the image data
        image_bytes = base64.b64decode(image_data)
        frame = Image.open(io.BytesIO(image_bytes))
        frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR)

        # Process the frame
        class_name, confidence_score_whole = drowsy_detector.process_frame(frame)

        # Add drowsiness details to the frame
        text = f"Class: {class_name}\nConfidence: {confidence_score_whole}%\nDrowsy Frames: {drowsy_detector.drowsy_frame_count}\nNon-Drowsy Frames: {drowsy_detector.non_drowsy_frame_count}"

        # Position the text on the right side of the frame
        text_x = frame.shape[1] - 300
        text_y = 50
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.8
        font_color = (0, 255, 0)
        font_thickness = 2

        # Split the text into lines
        lines = text.split('\n')
        for i, line in enumerate(lines):
            y = text_y + i * 30
            cv2.putText(frame, line, (text_x, y), font, font_scale, font_color, font_thickness, cv2.LINE_AA)

        # Display the result
        cv2_imshow(frame)
    except Exception as e:
        print(f"Error processing frame: {e}")

# Register callback function
output.register_callback('notebook.captureFrameCallback', capture_frame_callback)

# Start capturing
capture_image()

Failed to connect to the V2V server: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /socket.io/?transport=polling&EIO=4&t=1737869118.1620016 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7c98e412bd50>: Failed to establish a new connection: [Errno 111] Connection refused'))
